In [1]:
import os

In [2]:
%pwd

'e:\\github_projects\\Text_summarizer\\test_summerizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\github_projects\\Text_summarizer\\test_summerizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    #its a data class
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = Config_File_Path,
                 params_filepath = Params_File_Path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config['artifacts_root']])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']

        create_directories([config['root_dir']])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config['root_dir'],
            source_url = config['source_url'],
            local_data_file = config['local_data_file'],
            unzip_dir = config['unzip_dir']
        )

        return data_ingestion_config

        

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_file_size

In [9]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with these headers: \n{headers}")
        else:
            logger.info(f"File already exists with size of: {get_file_size(Path(self.config.local_data_file))}")

    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2025-07-17 01:07:20,236: INFO: common: path to yaml: config\config.yaml loaded successfully]
[2025-07-17 01:07:20,239: INFO: common: path to yaml: params.yaml loaded successfully]
✅ Created directory: artifacts
✅ Created directory: artifacts/data_ingestion
[2025-07-17 01:07:25,071: INFO: 1941293595: artifacts/data_ingestion/data.zip download with these headers: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A919:4A351:4678:F08B:6877FF71
Accept-Ranges: bytes
Date: Wed, 16 Jul 2025 19:37:23 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4729-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1752694643.547171,VS0,VE838
Vary: Authorization,Accept-E